In [1]:
import numpy as np
import networkx as nx
import scipy as sp
rng = np.random.default_rng()

from typing import List, Set, Dict, Tuple
from dataclasses import dataclass
from collections import defaultdict
from pathlib import Path

In [2]:
@dataclass
class DiffusionCascade:
  """Represents a single diffusion cascade through the network"""
  # List of sets, where D[t] contains nodes activated at time t
  D: List[List[int]]
  
  @property
  def T(self) -> int:
    """Returns the length of the cascade"""
    return len(self.D)
  
  def C(self, t: int) -> Set:
    """Returns the set of all nodes activated by time t"""
    return set().union(*self.D[:t+1]) if t >= 0 else set()

class ICModel:
    def __init__(self, G: nx.DiGraph, init_prob_range: Tuple[float, float] = (0.1, 0.3)):
      """
      Initialize the Independent Cascade Model using a NetworkX graph
      
      Args:
          G: NetworkX directed graph
          init_prob_range: Tuple of (min, max) for initial probability range
      """
      self.G = G
      self.nodes = set(G.nodes())
      self.edges = list(G.edges())
      
      # Create adjacency lists using NetworkX functionality
      self.children = {node: set(G.successors(node)) for node in G.nodes()}
      self.parents = {node: set(G.predecessors(node)) for node in G.nodes()}
          
      # Initialize diffusion probabilities randomly
      alpha, beta = init_prob_range
      self.kappa = {(v,w): np.random.uniform(alpha, beta) 
                    for v, w in self.edges}
      
      # Add probabilities as edge attributes in the graph
      nx.set_edge_attributes(self.G, self.kappa, 'diffusion_prob')

    def calculate_activation_probability(
      self, cascade: DiffusionCascade, w, t: int) -> float:
      """
      Calculate P_w(t+1) as defined in equation (3) of the paper
      
      Args:
          cascade: The diffusion cascade
          w: Target node
          t: Time step
      """
      # Get active parents of w at time t
      active_parents = self.parents[w].intersection(cascade.D[t])
      
      if not active_parents:
        return 0.0
          
      # Calculate 1 - ∏(1 - κ_v,w) for all active parents v
      prob = 1.0
      for v in active_parents:
        prob *= (1 - self.kappa[(v,w)])
      
      return 1.0 - prob

    def estimate_parameters(
      self, cascades: List[DiffusionCascade], max_iterations: int = 1000,
      tolerance: float = 1e-6, verbose: bool = True) -> Dict:
      """
      Implement the EM algorithm described in Section 2.3
      
      Args:
        cascades: List of observed diffusion cascades
        max_iterations: Maximum number of EM iterations
        tolerance: Convergence threshold for parameter changes
        verbose: Whether to print convergence information
      """
      iteration = 1
      while iteration <= max_iterations:
        if (iteration % 100 == 1 or iteration == max_iterations) and verbose:
          print(f"Iteration {iteration}")
        old_kappa = self.kappa.copy()
          
        # E-step: Calculate activation probabilities
        P_w = {}  # Store P_w^(s) for each cascade s and node w
        for s, cascade in enumerate(cascades):
          for t in range(cascade.T - 1):
            for w in cascade.D[t + 1]:
              P_w[(s,w)] = self.calculate_activation_probability(cascade, w, t)
          
        # M-step: Update κ_v,w using equation (8)
        for v, w in self.edges:
          # Calculate S^+_v,w and S^-_v,w as defined in the paper
          S_plus = []   # cascades where v activated w
          S_minus = []  # cascades where v failed to activate w
          
          for s, cascade in enumerate(cascades):
            for t in range(cascade.T - 1):
              if v in cascade.D[t]:
                if w in cascade.D[t + 1]:
                  S_plus.append((s,w))
                elif w not in cascade.C(t + 1):
                  S_minus.append((s,w))
          
          # Update κ_v,w using equation (8)
          if S_plus or S_minus:
            numerator = sum(self.kappa[(v,w)] / P_w[(s,w)] for s,w in S_plus)
            self.kappa[(v,w)] = numerator / (len(S_plus) + len(S_minus))
        
        # Update edge attributes in the graph
        nx.set_edge_attributes(self.G, self.kappa, 'diffusion_prob')
        
        # Check convergence
        max_change = max(abs(self.kappa[e] - old_kappa[e]) for e in self.kappa)
            
        #if verbose:
        #  print(f"Iteration {iteration + 1}, max parameter change: {max_change:.6f}")
            
        if max_change < tolerance:
          break
        iteration += 1
      
      if verbose:
        print(f"Converged after {iteration} iterations")
      return self.kappa

    def get_influence_probabilities(self) -> nx.DiGraph:
        """Returns a copy of the graph with diffusion probabilities as edge attributes"""
        return self.G.copy()

In [4]:
n = 100
p = 0.1
gname = f"er_{n}_{str(p).replace('.', '')}"
path = Path(f"datasets/synthetic/{gname}")

with open(path / f"{gname}.mtx", "rb") as fh:
    G = nx.from_scipy_sparse_array(sp.io.mmread(fh), create_using=nx.DiGraph)

cascades = []
#idxes = rng.choice(1000, 250, replace=False)
k = 250
for i in range(k):
  with open(path / f"diffusions/timestamps/{i}.txt", "r") as fh:
    cascade = []
    for line in fh:
        cascade.append(list(map(int, line.strip().split())))
    cascades.append(DiffusionCascade(cascade))


In [5]:
for i in (50, 100, 150, 200, 250):
    model = ICModel(G)
    # Example of analyzing the learned influence probabilities
    model.estimate_parameters(cascades[:i], tolerance=1e-6, verbose=False)
    print("")
    G_learned = model.get_influence_probabilities()
    residuals = []
    for edge in G.edges():
        u, v = edge
        true_prob = G[u][v]['weight']
        learned_prob = G_learned[u][v]['diffusion_prob']
        residuals.append(abs(true_prob - learned_prob))
        #print(f"Edge {edge}: True={true_prob:.3f}, Learned={learned_prob:.3f}")

    l1_error = sum(residuals) / len(residuals)
    #median = np.median(residuals)
    #quartile1 = np.percentile(residuals, 25) 
    #quartile3 = np.percentile(residuals, 75)

    print(f"MAE Error: {l1_error}")
    #print(f"Median: {median}, Q1: {quartile1}, Q3: {quartile3}")


MAE Error: 0.15410542297022634

MAE Error: 0.10029812531557204

MAE Error: 0.08086182880330223

MAE Error: 0.07120048577767397

MAE Error: 0.0627592441083516


In [46]:
G_learned = model.get_influence_probabilities()

residuals = []
for edge in G.edges():
  u, v = edge
  true_prob = G[u][v]['weight']
  learned_prob = G_learned[u][v]['diffusion_prob']
  residuals.append(abs(true_prob - learned_prob))
  #print(f"Edge {edge}: True={true_prob:.3f}, Learned={learned_prob:.3f}")

l1_error = sum(residuals) / len(residuals)
median = np.median(residuals)
quartile1 = np.percentile(residuals, 25) 
quartile3 = np.percentile(residuals, 75)

print(f"MAE Error: {l1_error}")
print(f"Median: {median}, Q1: {quartile1}, Q3: {quartile3}")

MAE Error: 0.10028962974039321
Median: 0.07677035581689905, Q1: 0.034881051799292226, Q3: 0.1443947388111413
